## **Testando integração com a API do Wisecare**

- formatar o json pra prescription. 
- pydantic class: Org Info e AppointmentTookPlaceIn, LLM structure output

In [1]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from pydantic import BaseModel, Field
from typing import List, Dict
from dotenv import load_dotenv
import os

# LOCAL IMPORTS
from agent import Doctor, AppointmentTookPlaceIn, LLMPrescription, Prescription, Doctor, SkinInfo
from agent import GENERATE_PRESCRIPTION_PROMPT
load_dotenv()  # Load environment variables from the .env file
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGSMITH_TRACING'] = os.getenv('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = os.getenv('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = os.getenv('LANGSMITH_PROJECT')

In [2]:
model = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)
model = model.with_structured_output(LLMPrescription)


**Exemplo 1**

In [ ]:
user_input = """Crie uma receita de Vonau de 8mg pra Roberta, genero feminino, 24 anos. Para tomar 3 vezes ao dia"""

prescription_prompt =  GENERATE_PRESCRIPTION_PROMPT.format(input = user_input , conversation_history = "")
prescription_prompt

In [4]:
prescription_response1 = model.invoke([SystemMessage(content= prescription_prompt)])

In [ ]:
json_prescription_1 = prescription_response1.dict()
json_prescription_1

**Exemplo 2**

In [ ]:
user_input = """Receita de Wellbutrin 20mg, tomar depois do cafe da manha, e Dipirona 10mg duas vezes ao dia. Paciente Kleber genero masculino, 30 anos."""

prescription_prompt2 =  GENERATE_PRESCRIPTION_PROMPT.format(input = user_input , conversation_history = "")
prescription_prompt2

In [ ]:
prescription_response2 = model.invoke([SystemMessage(content= prescription_prompt2)])
json_prescription_2 = prescription_response2.dict()
json_prescription_2

In [ ]:
prescription_response2

In [ ]:
json_prescription_2.get('consultant')
json_prescription_2.get('prescriptions')

## **MONTAR O PAYLOAD PRA API DA WISECARE**

In [ ]:
prescription_instance = Prescription(
    consultant = json_prescription_2.get('consultant'),
    prescriptions = json_prescription_2.get('prescriptions'),
    doctor = Doctor().model_dump(),
    appointmentTookPlaceIn = AppointmentTookPlaceIn().model_dump(),
)
prescription_instance

In [ ]:
prescription_instance = Prescription(
    consultant = json_prescription_2.get('consultant'),
    prescriptions = json_prescription_2.get('prescriptions'),
    doctor = Doctor().model_dump(),
    appointmentTookPlaceIn = AppointmentTookPlaceIn().model_dump(),
)

prescription_payload = prescription_instance.model_dump()

In [ ]:
prescription_payload = prescription_instance.model_dump()
prescription_payload

In [ ]:
import sys
import os
import datetime

wisecare_path = "/Users/robertagarcia/Desktop/learning/LangGraph/langgraph_personal/NodeInterrupt/medical-assistant/wisecare-prescription"
sys.path.append(wisecare_path)

# Import our prescription generator
from python_wrapper import generate_prescription

# Create a sample prescription for Filo Gomes
filo_prescription = {
    "prescription": {
        "codigo": "12345",
        "consultant": {
            "name": "Filo Gomes",
            "age": "42 anos",
            "gender": "Masculino"
        },
        "prescriptions": [{
            "name": "Losartana",
            "dosage": "50mg",
            "posology": "1 comprimido pela manhã"
        }, {
            "name": "Atorvastatina",
            "dosage": "20mg",
            "posology": "1 comprimido à noite"
        }],
        "doctor": {
            "name": "Médico Silva",
            "crm": "1231231",
            "uf": "PB"
        },
        "appointmentTookPlaceIn": {
            "name": "Nome",
            "address": "Rua das Flores, 137",
            "neighbourhood": "Bairro",
            "city": "Cidade",
            "uf": "PB",
            "phone": "110000000"
        }
    }
}

p = {'codigo': '12314',
 'consultant': {'name': 'Kleber', 'age': '30', 'gender': 'masculino'},
 'prescriptions': [{'name': 'Wellbutrin',
   'dosage': '20mg',
   'posology': 'tomar depois do café da manhã'},
  {'name': 'Dipirona', 'dosage': '10mg', 'posology': 'duas vezes ao dia'}],
 'doctor': {'name': 'Médico Silva', 'crm': '1231231', 'uf': 'PB'},
 'appointmentTookPlaceIn': {'name': 'Nome',
  'address': 'Rua das Flores, 137',
  'neighbourhood': 'Bairro',
  'city': 'Cidade',
  'uf': 'PB',
  'phone': '11999999999'}}

# Generate a PDF from our notebook
output_path = os.path.join(wisecare_path, "output/Kleber_prescription.pdf")
result = generate_prescription({"prescription" : p }, output_path)
print(f"Prescription for Filo Gomes generated at: {result}")

# You could also use prescription_payload from the notebook
# Just pass it to generate_prescription 
# result = generate_prescription({"prescription": prescription_payload}, output_path)

# **TEST WISECARE API GENERATE MEDICAL EXAM REQUEST**

In [ ]:
from agent import LLMExamRequest, ExamRequest
from agent import GENERATE_EXAM_REQUEST_PROMPT

In [5]:


model = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)
model = model.with_structured_output(LLMExamRequest)

In [7]:
user_input = "Solicito ultrassonografia de abdome total devido a dor abdominal no hipocôndrio direito. paciente = Jorge"
response = model.invoke( GENERATE_EXAM_REQUEST_PROMPT.format(input = user_input, conversation_history = ""))
print(response)

consultant='Jorge' clinicalIndication='Dor abdominal no hipocôndrio direito' request='Ultrassonografia de abdome total'


In [ ]:
import sys
import os
from datetime import datetime
from python_wrapper import generate_exam_request
# PYDANTIC CLASS FOR EMAM REQUEST PAYLOAD




# Add the path to the wisecare-prescription directory
wisecare_path = "/Users/robertagarcia/Desktop/learning/LangGraph/langgraph_personal/NodeInterrupt/medical-assistant/wisecare-prescription"
sys.path.append(wisecare_path)


In [13]:
skin_info = SkinInfo().model_dump()
skin_info

{'images': {'logo': 'https://minio.homolog.v4h.cloud/public/IntellidoctorLogo.png'},
 'colors': {'header': '#0A7BED',
  'footer': {'primary': '#0A7BED', 'secondary': '#054586'},
  'title': '#CF0014',
  'text': {'gray100': '#EFEFEF',
   'gray200': '#DBDBDB',
   'gray300': '#7A7A7A',
   'gray400': '#303030',
   'blue': '#0A7BED'},
  'background': '#ffffff'},
 'links': {'validationUrl': 'https://receita.v4h.cloud'}}

In [14]:

exam_request_payload = ExamRequest(
      consultant = response.model_dump().get("consultant"),
      clinicalIndication = response.model_dump().get("clinicalIndication"),
      request = response.model_dump().get("request"),
      doctor =  Doctor(),
      appointmentTookPlaceIn = AppointmentTookPlaceIn(),
      dateOfEmission = datetime.today().strftime("%d/%m/%Y")

)

exam_request_payload = exam_request_payload.model_dump()
exam_request_payload

{'consultant': 'Jorge',
 'codigo': 'EX12345',
 'clinicalIndication': 'Dor abdominal no hipocôndrio direito',
 'request': 'Ultrassonografia de abdome total',
 'doctor': {'name': 'Médico Silva', 'crm': '1231231', 'uf': 'PB'},
 'appointmentTookPlaceIn': {'name': 'Nome',
  'address': 'Rua das Flores, 137',
  'neighbourhood': 'Bairro',
  'city': 'Cidade',
  'uf': 'PB',
  'phone': '11999999999'},
 'dateOfEmission': '27/03/2025'}

In [15]:
output_path = os.path.join(wisecare_path, "output/jupyter_test_exam_request_2.pdf")
result = generate_exam_request(exam_request_payload, output_path, skin_info = skin_info)

Exam request generated and saved to /Users/robertagarcia/Desktop/learning/LangGraph/langgraph_personal/NodeInterrupt/medical-assistant/wisecare-prescription/output/jupyter_test_exam_request_2.pdf


In [ ]:
exam_request_payload = {
        "consultant": "Maria Silva",
        "codigo": "EX12345",
        "clinicalIndication": "Acompanhamento de rotina",
        "request": "Hemograma completo, Perfil lipídico, Glicemia de jejum, TSH, T4 livre",
        "doctor": {
            "name": "Dra. Ana Carvalho",
            "crm": "54321",
            "uf": "SP"
        },
        "appointmentTookPlaceIn": {
            "name": "Centro Médico Esperança",
            "address": "Avenida Paulista, 1000",
            "neighbourhood": "Bela Vista",
            "city": "São Paulo",
            "uf": "SP",
            "phone": "1198765432"
        },
        "dateOfEmission": "01/06/2023"
    }